## β叠加效应分析
根据[β模拟策略分析]()，对β叠加效应的研究分为以下几个过程：
1. 所有质子分支共享同一个Q值下β输入进行模拟
2. 所有质子共享按不同Q分支加权后的β输入进行模拟
3. 每个质子按照对应的Q分支作为β输入进行模拟

### 所有质子分支共享同一个Q值下β输入进行模拟
- 输入文件为beta.root, 选取Q=2MeV的β输入
- 质子和β+同时发射
- 粒子从后两块Si发射

In [1]:
%jsroot on
TFile *f1=new TFile("simualsp.root");
TFile *f2=new TFile("simualspbeta.root");
TCanvas c;

In [2]:
f1->cd();
tree->Draw("de[1]>>p010(2000,0,8)","de[2]<0.01&&de[0]<0.01&&de[1]>0");
TH1D *p010=(TH1D*)f1->Get("p010");

In [3]:
f2->cd();
tree->Draw("de[1]>>pb010(2000,0,8)","de[2]<0.01&&de[0]<0.01&&de[1]>0");
TH1D *pb010=(TH1D*)f2->Get("pb010");

In [4]:
c.Clear();
p010->SetLineColor(1);
pb010->SetLineColor(2);
p010->Draw();
pb010->Draw("same");
c.Draw();

In [5]:
f1->cd();
tree->Draw("de[2]>>p001(2000,0,8)","de[1]<0.01&&de[0]<0.01&&de[2]>0");
TH1D *p001=(TH1D*)f1->Get("p001");

In [6]:
f2->cd();
tree->Draw("de[2]>>pb001(2000,0,8)","de[1]<0.01&&de[0]<0.01&&de[2]>0");
TH1D *pb001=(TH1D*)f2->Get("pb001");

In [7]:
c.Clear();
p001->SetLineColor(1);
pb001->SetLineColor(2);
p001->Draw();
pb001->Draw("same");
c.Draw();

In [8]:
f1->Close();
f2->Close();

发现似乎并不能看到β叠加效应。似乎能谱中只保留了质子部分。

注册粒子枪部分代码如下：
```cpp
DefineParticleGun(maxhit,"proton",G4ThreeVector(x*um,y*um,z*um),G4RandomDirection(), pe*keV);
fParticleGun->GeneratePrimaryVertex(event);
DefineParticleGun(maxhit,"e+",G4ThreeVector(x*um,y*um,z*um),G4RandomDirection(), be*GeV);
fParticleGun->GeneratePrimaryVertex(event); 
```

更改注册粒子枪顺序：
```cpp
DefineParticleGun(maxhit,"e+",G4ThreeVector(x*um,y*um,z*um),G4RandomDirection(), be*GeV);
fParticleGun->GeneratePrimaryVertex(event); 
DefineParticleGun(maxhit,"proton",G4ThreeVector(x*um,y*um,z*um),G4RandomDirection(), pe*keV);
fParticleGun->GeneratePrimaryVertex(event);
```

In [9]:
c.Clear();
TFile *f3=new TFile("simualspbeta2.root");
f3->cd();
tree->Draw("de[1]","de[1]>0");
c.Draw();

In [10]:
f3->Close();

会发现调整注册粒子枪顺序之后，探测到的仅仅为β粒子。
上图为注册两个粒子、下图为注册一个粒子，可以看出注册粒子是成功的，但数据异常。
![](https://i.loli.net/2019/07/03/5d1cc7c8c136196061.png)
![](https://i.loli.net/2019/07/03/5d1cc7f7ee83c64430.png)

这个原因可能在于在`SensitiveDetector.cc`中的`ProcessHits`函数的处理方式。
```cpp
G4bool SensitiveDetector::ProcessHits(G4Step *step, G4TouchableHistory *history)
{
    int currentTrackID=step->GetTrack()->GetTrackID();
    G4double edep=step->GetTotalEnergyDeposit();
    if(edep==0.) return  false;
    auto touchable=step->GetPreStepPoint()->GetTouchableHandle();
    int copyid=touchable->GetCopyNumber();
    int xid=copyid/16;
    int yid=copyid%16;
    SDHit *aHit=nullptr;
    for (int i = 0; i < fHitsCollection->entries(); ++i) {
        if((*fHitsCollection)[i]->GetTrackID()==currentTrackID &&
                (*fHitsCollection)[i]->GetCopyID()==copyid){
            aHit=(*fHitsCollection)[i];
        }
    }
    if(!aHit){
        aHit=new SDHit();
        fHitsCollection->insert(aHit);
        aHit->SetTrackID(step);
        aHit->SetEventID();
        aHit->SetXid(xid);
        aHit->SetYid(yid);
        aHit->SetCopyID(copyid);
        aHit->SetIsHit(true);
    }
    aHit->AddEdep(edep);
    aHit->SetPos(step);
    return true;
}
```

把调试开关打开，输出
```cpp
void SensitiveDetector::EndOfEvent(G4HCofThisEvent *hitCollection)
{

// if(verboseLevel>1){
        G4int nofHits=fHitsCollection->entries();
        G4cout<<G4endl
             <<"------->Hits Collection: in this event they are "<<nofHits
            <<" hits in the tracker chambers: "<<G4endl;
        for (G4int i=0;i<nofHits;i++) {
            if((*fHitsCollection)[i]->GetIsHit())
                G4cout<<(*fHitsCollection)[i]->GetEventID()<<'\t'
                        <<(*fHitsCollection)[i]->GetTrackID()<<'\t'
                      <<(*fHitsCollection)[i]->GetXid()<<'\t'
                   <<(*fHitsCollection)[i]->GetEventID()<<'\t'    <<(*fHitsCollection)[i]->GetYid()<<'\t'
                       <<(*fHitsCollection)[i]->GetEdep()<<'\t'
                       <<G4endl;
        }
// }
}
```
发现质子和电子事件均可以输出，两者是同一个event，TrackID分为为1和0，这和注册粒子枪的顺序是有关系的。先注册的那个为0，后注册的为1.

调整程序结构，保留质子和β事件。

In [11]:
TFile *f4=new TFile("simualspbeta3.root");
tree->Draw("de[1][0]>>b1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0");
tree->Draw("de[1][1]>>p1_010(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0");
tree->Draw("de[1][0]+de[1][1]>>bp1_010(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0");

tree->Draw("de[2][0]>>b2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0");
tree->Draw("de[2][1]>>p2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0");
tree->Draw("de[2][0]+de[2][1]>>bp2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0");

In [12]:
c.Clear();
b1_010->SetLineColor(1);
p1_010->SetLineColor(2);
b1_010->Draw();
p1_010->Draw("same");
bp1_010->Draw("same");
c.Draw();

In [13]:
cout<<"proton counts at 731keV : "<<p1_010->Integral(85,160)<<'\n'
    <<"proton counts at 731kev with beta : "<<bp1_010->Integral(85,160)<<'\n';

proton counts at 731keV : 78414
proton counts at 731kev with beta : 75266


加上$\beta$谱之后，峰位整体右移，同时峰的sigma变大。

In [14]:
c.Clear();
b2_001->SetLineColor(1);
p2_001->SetLineColor(2);
b2_001->Draw();
p2_001->Draw("same");
bp2_001->Draw("same");
c.Draw();

加上$\beta$谱之后，峰位整体右移，同时在每个小峰之后，还有一个伴峰。看β谱会发现，也是分为两部分。猜测是$\beta$粒子前后方向发射的原因，下面做印证：

In [16]:
c.Clear();
tree->Draw("de[2][0]","abs(dz[2][0]-38)<0.01");
c.Draw();

In [17]:
c.Clear();
tree->Draw("de[2][0]","abs(dz[2][0]-38.304)<0.01");
c.Draw();

In [18]:
c.Clear();
tree->Draw("de[2][0]+de[2][1]>>bp2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&abs(dz[2][0]-38)<0.01");
c.Draw();

In [19]:
c.Clear();
tree->Draw("de[2][1]>>p2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&abs(dz[2][0]-38)<0.01");
c.Draw();

In [20]:
c.Clear();
tree->Draw("de[2][0]+de[2][1]>>bp2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&abs(dz[2][0]-38.304)<0.01");
c.Draw();

In [21]:
c.Clear();
tree->Draw("de[2][1]>>p2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&abs(dz[2][0]-38.304)<0.01");
c.Draw();